In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import transformers
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer

In [1]:
pwd

'C:\\Users\\Karthiek Duggirala\\Downloads'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# load dataframe
path = '/content/drive/MyDrive/preprocessed_data.csv'
df = pd.read_csv(path)

In [ ]:
# Encode the genre labels
label_encoder = LabelEncoder()
df['genre_code'] = label_encoder.fit_transform(df['genre'])

In [ ]:
# Split the dataset into training and testing sets
train_data, test_data, train_labels, test_labels = train_test_split(df['lyrics'], df['genre_code'], test_size=0.2, stratify = df['genre'] ,random_state=42)

In [ ]:
# Initialize TfidfVectorizer with a maximum of 10000 features
vectorizer = TfidfVectorizer(max_features=2000)

# Fit and transform the training data
train_vectors = vectorizer.fit_transform(train_data)

# Transform the testing data
test_vectors = vectorizer.transform(test_data)

In [ ]:
# Convert the numerical vectors into dense tensors
train_vectors = tf.convert_to_tensor(train_vectors.toarray(), dtype=tf.float32)
test_vectors = tf.convert_to_tensor(test_vectors.toarray(), dtype=tf.float32)

In [ ]:
# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_vectors, train_labels)).shuffle(len(train_data)).batch(16)
test_dataset = tf.data.Dataset.from_tensor_slices((test_vectors, test_labels)).batch(16)

In [ ]:
# Define the model architecture with LSTM
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(2000, 128, input_length=train_vectors.shape[1]))
model.add(tf.keras.layers.LSTM(units=128, dropout=0.4, recurrent_dropout=0.2))
model.add(tf.keras.layers.Dense(3, activation='softmax'))

In [ ]:
# Define the optimizer, loss function and evaluation metric
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy()
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')


In [ ]:
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [1]:
# Train the model
history = model.fit(train_dataset, epochs=10, validation_data=test_dataset)

Epoch 1/10
 3227/3227 [==============================] - 79s 236ms/step - loss: 0.9079 - accuracy: 0.5462 - val_loss: 0.7748 - val_accuracy: 0.6273
Epoch 2/10
 3227/3227 [==============================] - 75s 232ms/step - loss: 0.7721 - accuracy: 0.6672 - val_loss: 0.7448 - val_accuracy: 0.6373
Epoch 3/10
 3227/3227 [==============================] - 74s 230ms/step - loss: 0.7571 - accuracy: 0.6891 - val_loss: 0.7367 - val_accuracy: 0.6429
Epoch 4/10
 3227/3227 [==============================] - 76s 234ms/step - loss: 0.7434 - accuracy: 0.7063 - val_loss: 0.7264 - val_accuracy: 0.6602
Epoch 5/10
 3227/3227 [==============================] - 75s 232ms/step - loss: 0.7079 - accuracy: 0.7462 - val_loss: 0.7146 - val_accuracy: 0.6674
Epoch 6/10
 3227/3227 [==============================] - 75s 232s/step - loss: 0.6878 - accuracy: 0.7889 -  val_loss: 0.7028 - val_accuracy: 0.6812
Epoch 7/10
 3227/3227 [==============================] - 75s 232ms/step - loss: 0.6629 - accuracy: 0.7922 - val_

In [2]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_data, test_labels)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

864/864 [==============================] - 11s 26ms/step - loss: 0.6811 - accuracy: 0.6939
Test loss: 0.681183712263828
Test accuracy: 0.6939717835841186
